In [1]:
EXPT_NAME="Exp03b: Multi-Effect Evaporator"
import pandas as pd
import numpy as np
import sympy as sp
import math

import platform
from datetime import datetime
print("**************************************")
print(EXPT_NAME)
print("Roll No: 120CH0057, Name: Siddharth Gautam")
print("Running On: Python ", platform.python_version(), " on: ", str(datetime.now()))
print("**************************************")
print("")

# Mass flow in kg/s
MFR = 10000 / 3600
# initial conc
c_i = 0.1
# final conc
c_f = 0.5
# Steam pressure in kilopascals
steam_pressure = 2 * (10 ** 5)
# 100 mm vacuum
last_effect_pressure = 101325*(1 - (100/760)) 
U1 = 3400
U2 = 1420
U3 = 710
Cp = 4186
BPE = 0
TF = 40

steam_table = pd.read_csv("steam-table.csv")
pressure = steam_table["pressure"]
boiling_point = steam_table["temperature"]
H = steam_table["enthalpy_gas"]
V = steam_table["vaporization"]

def get_saturation_temperature(P):
    return np.interp(P * (10**(-5)), pressure, boiling_point)

def get_enthalpy(T):
    return np.interp(T, boiling_point, H)

def get_Hvap(T):
    return np.interp(T, boiling_point, V)

def get_pressure(T_sat):
    return np.interp(T_sat, boiling_point, pressure)

# Get steam temperature
TS = get_saturation_temperature(steam_pressure)
print("Steam temperature: " + str(TS) + " C")
# Get temperature in last effect
T3 = get_saturation_temperature(last_effect_pressure)
print("Last effect temperature: " + str(T3) + " C")
dT = T3 - TF
print("dT: " + str(dT) + " C")

# U1 * A1 * dT1 = U2 * A2 * dT2 = U3 * A3 * dT3
# Assumption = A1 = A2 = A3
# We solve the system
# dT1 + dT2 + dT3 - dT = 0
# U1*dT1 - U2*dT2 = 0
# U1*dT1 - U3*dT3 = 0

dT1, dT2, dT3 = sp.symbols('dT1, dT2, dT3', real=True, positive=True)
result = sp.solve([dT1 + dT2 + dT3 - dT, U1*dT1 - U2*dT2, U1*dT1 - U3*dT3], [dT1, dT2, dT3], dict=True)
dT1 = result[0][dT1]
dT2 = result[0][dT2]
dT3 = result[0][dT3]

T1 = TF + dT1
T2 = T1 + dT2
T3 = T2 + dT3

P1 = get_pressure(T1)
P2 = get_pressure(T2)
P3 = get_pressure(T3)

print("Temperature in Effect 1: " + str(T1) + " C")
print("Temperature in Effect 2: " + str(T2) + " C")
print("Temperature in Effect 3: " + str(T3) + " C")

HF = TF * Cp / 1000
H1 = T1 * Cp / 1000
H2 = T2 * Cp / 1000
H3 = T3 * Cp / 1000

lambda_S = get_Hvap(TS)
print("Steam Enthalpy: " + str(lambda_S) + " kJ/kg")

HV1 = get_enthalpy(T1)
lambda_V1 = get_Hvap(T1)

HV2 = get_enthalpy(T2)
lambda_V2 = get_Hvap(T2)

HV3 = get_enthalpy(T3)
lambda_V3 = get_Hvap(T3)

print("H1: " + str(H1) + " H_V1: " + str(HV1) + " lambda_V1: " + str(lambda_V1))
print("H2: " + str(H2) + " H_V2: " + str(HV2) + " lambda_V2: " + str(lambda_V2))
print("H3: " + str(H3) + " H_V3: " + str(HV3) + " lambda_V3: " + str(lambda_V3))

# Find MFR of last effect by using material balance
m3 = c_i * MFR / c_f
mv = MFR - m3

print("Total vapor created: " + str(mv) + " kg/s")

m1, m2, ms, mv1, mv2, mv3 = sp.symbols('ms, m1, m2, mv1, mv2, mv3', real=True, positive=True)

# Create the system of equations
system = [mv1 + mv2 + mv3 - mv, (MFR - mv1 - m1), ms*lambda_S + MFR * HF - (mv1*HV1 + m1*H1), m1 - (mv2 + m2), m1*H1 + mv1*lambda_V1 - (mv2*HV2 + m2*H2), m2*H2+mv2*lambda_V2 - (mv3*HV3 + m3*H3)]
result = sp.solve(system, [m1, m2, ms, mv1, mv2, mv3], dict=True)

m1 = result[0][m1]
m2 = result[0][m2]
ms = result[0][ms]
mv1 = result[0][mv1]
mv2 = result[0][mv2]
mv3 = result[0][mv3]

print("Liquid flow out of effect 1: " + str(m1) + " kg/s")
print("Liquid flow out of effect 2: " + str(m2) + " kg/s")
print("Liquid flow out of effect 3: " + str(m3) + " kg/s")
print("Vapor flow out of effect 1: " + str(mv1) + " kg/s")
print("Vapor flow out of effect 2: " + str(mv2) + " kg/s")
print("Vapor flow out of effect 3: " + str(mv3) + " kg/s")
print("Flow rate of Steam: " + str(ms) + " kg/s")


A1 = (ms*lambda_S / (U1 * dT1))*1000
A2 = (mv1*lambda_V1 / (U2 * dT2))*1000
A3 = (mv2*lambda_V2 / (U3 * dT3))*1000
A_av = (A1 + A2 + A3) / 3.0

steam_economy = (mv1 + mv2 + mv3) / ms
print("Steam Economy: " + str(steam_economy))

print("Area of Effect 1: " + str(A1) + " m^2")
print("Area of Effect 2: " + str(A2) + " m^2")
print("Area of Effect 3: " + str(A3) + " m^2")

ErrA = math.sqrt((A1-A2)**2 + (A2-A3)**2 + (A1-A3)**2) / 3
Percent_ErrA = round((ErrA/A_av)*100,2)
print("% Error in area: " + str(Percent_ErrA) + " %")

if(Percent_ErrA < 10.0):
    print("Error is within bounds. Design successful!")
else:
    print("Design failed. Reset parameters.")
    

result = { "Steam flow (kg/s)": ms, "Pressure in E1": P1, "Pressure in E2": P2, "Pressure in E3": P3, "Area E1 (m^2)": A1, "Area E2 (m^2)": A2, "Area E3 (m^2)": A3, "Error (%) [OK]" if Percent_ErrA < 10.0 else "Error (%) [NOT OK]": Percent_ErrA, "Steam Economy" : steam_economy }
import pandas as pd
df = pd.DataFrame(result.items(), columns=['Parameter', 'Value'])
display(df)

**************************************
Exp03b: Multi-Effect Evaporator
Roll No: 120CH0057, Name: Siddharth Gautam
Running On: Python  3.11.1  on:  2023-04-25 15:31:48.460061
**************************************

Steam temperature: 120.2 C
Last effect temperature: 96.08020642201835 C
dT: 56.08020642201835 C
Temperature in Effect 1: 46.8531749672346 C
Temperature in Effect 2: 63.2621854521625 C
Temperature in Effect 3: 96.0802064220183 C
Steam Enthalpy: 2201.9 kJ/kg
H1: 196.127390412844 H_V1: 2586.5423615173117 lambda_V1: 2390.2854254966423
H2: 264.815508302752 H_V2: 2615.2746548357486 lambda_V2: 2350.4565902892377
H3: 402.191744082569 H_V3: 2669.8687590535974 lambda_V3: 2267.357351521004
Total vapor created: 2.2222222222222223 kg/s
Liquid flow out of effect 1: 1.99094488221220 kg/s
Liquid flow out of effect 2: 1.24896178994597 kg/s
Liquid flow out of effect 3: 0.5555555555555556 kg/s
Vapor flow out of effect 1: 0.786832895565573 kg/s
Vapor flow out of effect 2: 0.741983092266234 kg/s


,Parameter,Value
0,Steam flow (kg/s),0.890387542081228
1,Pressure in E1,0.105851
2,Pressure in E2,0.232605
3,Pressure in E3,0.879928
4,Area E1 (m^2),84.1406629379867
5,Area E2 (m^2),80.7163537365873
6,Area E3 (m^2),74.8471911554311
7,Error (%) [OK],4.80
8,Steam Economy,2.49579213229771
